# LME銅先物 隣月間スプレッド 基本分析

## 概要
このノートブックでは、LME銅先物の隣月間スプレッド（M1-M2、M2-M3、M3-M4）の基本分析を行います。

### 分析対象
- **M1-M2スプレッド**: 第1限月 - 第2限月
- **M2-M3スプレッド**: 第2限月 - 第3限月  
- **M3-M4スプレッド**: 第3限月 - 第4限月

### 期待される成果
- 隣月間スプレッドの基本統計量と分布特性
- 各スプレッドの時系列推移とトレンド
- スプレッド水準の季節性とパターン
- リスク特性とボラティリティ分析

In [ ]:
# 必要ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sqlalchemy import create_engine
import warnings
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
import os

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# データベース接続設定
def get_db_connection():
    """PostgreSQLデータベースへの接続を取得"""
    try:
        engine = create_engine('postgresql://Yusuke@localhost:5432/lme_copper_db')
        return engine
    except Exception as e:
        print(f"データベース接続エラー: {e}")
        return None

# データベース接続テスト
engine = get_db_connection()
if engine:
    print("✅ データベース接続成功")
else:
    print("❌ データベース接続失敗")

## 1. データ取得と前処理

In [ ]:
def load_futures_data():
    """先物データの取得"""
    query = """
    SELECT 
        trade_date,
        contract_month,
        close_price,
        volume,
        open_interest,
        ric
    FROM lme_copper_futures 
    WHERE contract_month IN (1, 2, 3, 4)
        AND close_price IS NOT NULL
        AND close_price > 0
    ORDER BY trade_date, contract_month
    """
    
    df = pd.read_sql(query, engine)
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    
    print(f"📊 データ取得完了:")
    print(f"   総レコード数: {len(df):,}")
    print(f"   データ期間: {df['trade_date'].min()} ～ {df['trade_date'].max()}")
    print(f"   限月別レコード数:")
    for month in [1, 2, 3, 4]:
        count = len(df[df['contract_month'] == month])
        print(f"     M{month}: {count:,} レコード")
    
    return df

# データ取得
futures_data = load_futures_data()

In [ ]:
def calculate_adjacent_spreads(df):
    """隣月間スプレッドの計算"""
    # データをピボットして限月別カラムに変換
    pivot_df = df.pivot(index='trade_date', columns='contract_month', values='close_price')
    
    # カラム名を分かりやすく変更
    pivot_df.columns = [f'M{int(col)}' for col in pivot_df.columns]
    
    # 隣月間スプレッドを計算
    spreads_df = pd.DataFrame(index=pivot_df.index)
    
    spreads_df['M1_price'] = pivot_df['M1']
    spreads_df['M2_price'] = pivot_df['M2']
    spreads_df['M3_price'] = pivot_df['M3']
    spreads_df['M4_price'] = pivot_df['M4']
    
    # スプレッドの計算（前限月 - 後限月）
    spreads_df['M1_M2_spread'] = pivot_df['M1'] - pivot_df['M2']
    spreads_df['M2_M3_spread'] = pivot_df['M2'] - pivot_df['M3']
    spreads_df['M3_M4_spread'] = pivot_df['M3'] - pivot_df['M4']
    
    # スプレッド変化率（日次リターン）
    spreads_df['M1_M2_spread_change'] = spreads_df['M1_M2_spread'].pct_change()
    spreads_df['M2_M3_spread_change'] = spreads_df['M2_M3_spread'].pct_change()
    spreads_df['M3_M4_spread_change'] = spreads_df['M3_M4_spread'].pct_change()
    
    # NaN値を除去
    spreads_df = spreads_df.dropna()
    
    print(f"📈 スプレッド計算完了:")
    print(f"   有効データ数: {len(spreads_df):,}")
    print(f"   欠損値除去後期間: {spreads_df.index.min()} ～ {spreads_df.index.max()}")
    
    return spreads_df

# スプレッド計算
spreads_data = calculate_adjacent_spreads(futures_data)

# データの先頭確認
print("\n📋 計算されたスプレッドデータ（直近5日分）:")
print(spreads_data.tail())

## 2. 基本統計量とデータ概要

In [ ]:
def calculate_basic_statistics(df):
    """基本統計量の計算"""
    spread_columns = ['M1_M2_spread', 'M2_M3_spread', 'M3_M4_spread']
    
    stats_df = pd.DataFrame()
    
    for col in spread_columns:
        spread_name = col.replace('_spread', '').replace('_', '-')
        
        stats_df[spread_name] = [
            df[col].count(),           # 有効データ数
            df[col].mean(),            # 平均
            df[col].median(),          # 中央値
            df[col].std(),             # 標準偏差
            df[col].min(),             # 最小値
            df[col].max(),             # 最大値
            df[col].quantile(0.25),    # 第1四分位数
            df[col].quantile(0.75),    # 第3四分位数
            df[col].skew(),            # 歪度
            df[col].kurtosis(),        # 尖度
        ]
    
    stats_df.index = ['データ数', '平均', '中央値', '標準偏差', '最小値', '最大値', 
                      'Q1(25%)', 'Q3(75%)', '歪度', '尖度']
    
    return stats_df

# 基本統計量を計算
basic_stats = calculate_basic_statistics(spreads_data)

print("📊 隣月間スプレッド基本統計量:")
print("=" * 50)
print(basic_stats.round(4))

In [ ]:
# スプレッドの分布特性を詳細分析
def analyze_spread_characteristics(df):
    """スプレッド特性の詳細分析"""
    spread_columns = ['M1_M2_spread', 'M2_M3_spread', 'M3_M4_spread']
    
    print("\n🔍 隣月間スプレッド特性分析:")
    print("=" * 60)
    
    for col in spread_columns:
        spread_name = col.replace('_spread', '').replace('_', '-')
        data = df[col]
        
        # 正値・負値の分布
        positive_ratio = (data > 0).mean() * 100
        negative_ratio = (data < 0).mean() * 100
        zero_ratio = (data == 0).mean() * 100
        
        # 極値の頻度
        q95 = data.quantile(0.95)
        q05 = data.quantile(0.05)
        extreme_high = (data > q95).sum()
        extreme_low = (data < q05).sum()
        
        print(f"\n{spread_name}スプレッド:")
        print(f"  正値（コンタンゴ）: {positive_ratio:.1f}%")
        print(f"  負値（バックワーデーション）: {negative_ratio:.1f}%")
        print(f"  ゼロ（フラット）: {zero_ratio:.1f}%")
        print(f"  95%分位点超過: {extreme_high}回 ({extreme_high/len(data)*100:.1f}%)")
        print(f"  5%分位点未満: {extreme_low}回 ({extreme_low/len(data)*100:.1f}%)")
        print(f"  変動幅（Max-Min）: {data.max() - data.min():.2f} USD/t")

analyze_spread_characteristics(spreads_data)

## 3. 時系列推移とトレンド分析

In [ ]:
# 隣月間スプレッドの時系列チャート
def plot_spread_timeseries(df):
    """隣月間スプレッドの時系列チャート"""
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('M1-M2スプレッド', 'M2-M3スプレッド', 'M3-M4スプレッド'),
        vertical_spacing=0.08
    )
    
    spreads = [
        ('M1_M2_spread', 'M1-M2', 'blue'),
        ('M2_M3_spread', 'M2-M3', 'red'),
        ('M3_M4_spread', 'M3-M4', 'green')
    ]
    
    for i, (col, name, color) in enumerate(spreads, 1):
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df[col],
                name=f"{name}スプレッド",
                line=dict(color=color, width=1),
                mode='lines'
            ),
            row=i, col=1
        )
        
        # ゼロラインを追加
        fig.add_hline(
            y=0, 
            line_dash="dash", 
            line_color="black", 
            line_width=1,
            row=i, col=1
        )
    
    fig.update_layout(
        title=dict(
            text="LME銅先物 隣月間スプレッド時系列推移",
            x=0.5,
            font=dict(size=16)
        ),
        height=800,
        showlegend=False
    )
    
    # Y軸ラベル
    for i in range(1, 4):
        fig.update_yaxes(title_text="スプレッド (USD/t)", row=i, col=1)
    
    fig.update_xaxes(title_text="日付", row=3, col=1)
    
    return fig

# チャート表示
spread_chart = plot_spread_timeseries(spreads_data)
spread_chart.show()

# 画像保存
os.makedirs('../generated_images', exist_ok=True)
spread_chart.write_image('../generated_images/adjacent_spreads_timeseries.png', 
                        width=1200, height=800, scale=2)

In [ ]:
# スプレッドの移動平均とトレンド分析
def calculate_trend_analysis(df):
    """トレンド分析とテクニカル指標"""
    spread_columns = ['M1_M2_spread', 'M2_M3_spread', 'M3_M4_spread']
    
    trend_df = df.copy()
    
    for col in spread_columns:
        # 移動平均
        trend_df[f'{col}_ma5'] = df[col].rolling(window=5).mean()
        trend_df[f'{col}_ma20'] = df[col].rolling(window=20).mean()
        trend_df[f'{col}_ma60'] = df[col].rolling(window=60).mean()
        
        # ボリンジャーバンド（20日、2σ）
        ma20 = trend_df[f'{col}_ma20']
        std20 = df[col].rolling(window=20).std()
        trend_df[f'{col}_bb_upper'] = ma20 + (std20 * 2)
        trend_df[f'{col}_bb_lower'] = ma20 - (std20 * 2)
        
        # RSI（14日）
        delta = df[col].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        trend_df[f'{col}_rsi'] = 100 - (100 / (1 + rs))
    
    return trend_df

# トレンド分析実行
trend_data = calculate_trend_analysis(spreads_data)

print("📈 トレンド分析指標を計算しました:")
print(f"   移動平均: 5日、20日、60日")
print(f"   ボリンジャーバンド: 20日±2σ")
print(f"   RSI: 14日")

In [ ]:
# M1-M2スプレッドのテクニカル分析チャート
def plot_technical_analysis(df, spread_col='M1_M2_spread', title='M1-M2スプレッド'):
    """テクニカル分析チャート"""
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.7, 0.3],
        subplot_titles=(f'{title} + テクニカル指標', 'RSI'),
        vertical_spacing=0.05
    )
    
    # メインチャート（スプレッド + 移動平均 + ボリンジャーバンド）
    fig.add_trace(
        go.Scatter(
            x=df.index[-252:],  # 直近1年分
            y=df[spread_col].iloc[-252:],
            name=title,
            line=dict(color='blue', width=1.5)
        ),
        row=1, col=1
    )
    
    # 移動平均
    fig.add_trace(
        go.Scatter(
            x=df.index[-252:],
            y=df[f'{spread_col}_ma20'].iloc[-252:],
            name='MA20',
            line=dict(color='orange', width=1)
        ),
        row=1, col=1
    )
    
    # ボリンジャーバンド
    fig.add_trace(
        go.Scatter(
            x=df.index[-252:],
            y=df[f'{spread_col}_bb_upper'].iloc[-252:],
            name='BB Upper',
            line=dict(color='red', width=1, dash='dash'),
            showlegend=False
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index[-252:],
            y=df[f'{spread_col}_bb_lower'].iloc[-252:],
            name='BB Lower',
            line=dict(color='red', width=1, dash='dash'),
            fill='tonexty',
            fillcolor='rgba(255,0,0,0.1)',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # RSI
    fig.add_trace(
        go.Scatter(
            x=df.index[-252:],
            y=df[f'{spread_col}_rsi'].iloc[-252:],
            name='RSI',
            line=dict(color='purple', width=1.5)
        ),
        row=2, col=1
    )
    
    # RSI基準線
    fig.add_hline(y=70, line_dash="dash", line_color="red", line_width=1, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="blue", line_width=1, row=2, col=1)
    fig.add_hline(y=50, line_dash="dot", line_color="gray", line_width=1, row=2, col=1)
    
    # ゼロライン
    fig.add_hline(y=0, line_dash="dash", line_color="black", line_width=1, row=1, col=1)
    
    fig.update_layout(
        title=dict(
            text=f"LME銅先物 {title} テクニカル分析（直近1年）",
            x=0.5,
            font=dict(size=16)
        ),
        height=600,
        showlegend=True
    )
    
    fig.update_yaxes(title_text="スプレッド (USD/t)", row=1, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=1, range=[0, 100])
    fig.update_xaxes(title_text="日付", row=2, col=1)
    
    return fig

# M1-M2スプレッドのテクニカル分析
tech_chart = plot_technical_analysis(trend_data, 'M1_M2_spread', 'M1-M2スプレッド')
tech_chart.show()

# 画像保存
tech_chart.write_image('../generated_images/m1_m2_spread_technical_analysis.png', 
                      width=1200, height=600, scale=2)

## 4. スプレッド分布とリスク分析

In [ ]:
# スプレッド分布の詳細分析
def plot_spread_distributions(df):
    """スプレッド分布の可視化"""
    spread_columns = ['M1_M2_spread', 'M2_M3_spread', 'M3_M4_spread']
    spread_names = ['M1-M2', 'M2-M3', 'M3-M4']
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'スプレッド分布（ヒストグラム）',
            'スプレッド分布（ボックスプロット）',
            'Q-Qプロット（正規性検定）',
            'スプレッド相関行列'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    colors = ['blue', 'red', 'green']
    
    # 1. ヒストグラム
    for i, (col, name, color) in enumerate(zip(spread_columns, spread_names, colors)):
        fig.add_trace(
            go.Histogram(
                x=df[col],
                name=f'{name}スプレッド',
                nbinsx=50,
                opacity=0.7,
                marker_color=color
            ),
            row=1, col=1
        )
    
    # 2. ボックスプロット
    for i, (col, name, color) in enumerate(zip(spread_columns, spread_names, colors)):
        fig.add_trace(
            go.Box(
                y=df[col],
                name=f'{name}',
                marker_color=color,
                boxpoints='outliers'
            ),
            row=1, col=2
        )
    
    # 3. Q-Qプロット（M1-M2スプレッドのみ）
    from scipy.stats import probplot
    theoretical_quantiles, sample_quantiles = probplot(df['M1_M2_spread'].dropna(), dist="norm")
    
    fig.add_trace(
        go.Scatter(
            x=theoretical_quantiles,
            y=sample_quantiles,
            mode='markers',
            name='M1-M2 Q-Q',
            marker=dict(color='blue', size=4)
        ),
        row=2, col=1
    )
    
    # 理論線（正規分布）
    fig.add_trace(
        go.Scatter(
            x=[theoretical_quantiles.min(), theoretical_quantiles.max()],
            y=[sample_quantiles.min(), sample_quantiles.max()],
            mode='lines',
            name='理論線',
            line=dict(color='red', dash='dash')
        ),
        row=2, col=1
    )
    
    # 4. 相関行列のヒートマップ
    corr_matrix = df[spread_columns].corr()
    
    fig.add_trace(
        go.Heatmap(
            z=corr_matrix.values,
            x=spread_names,
            y=spread_names,
            colorscale='RdBu',
            zmid=0,
            text=np.round(corr_matrix.values, 3),
            texttemplate="%{text}",
            textfont={"size": 12},
            showscale=True
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        title=dict(
            text="隣月間スプレッド分布・リスク分析",
            x=0.5,
            font=dict(size=16)
        ),
        height=800,
        showlegend=True
    )
    
    # 軸ラベル更新
    fig.update_xaxes(title_text="スプレッド (USD/t)", row=1, col=1)
    fig.update_yaxes(title_text="頻度", row=1, col=1)
    fig.update_yaxes(title_text="スプレッド (USD/t)", row=1, col=2)
    fig.update_xaxes(title_text="理論分位数", row=2, col=1)
    fig.update_yaxes(title_text="サンプル分位数", row=2, col=1)
    
    return fig

# 分布分析チャート
dist_chart = plot_spread_distributions(spreads_data)
dist_chart.show()

# 画像保存
dist_chart.write_image('../generated_images/adjacent_spreads_distribution_analysis.png', 
                      width=1200, height=800, scale=2)

In [ ]:
# 正規性検定と統計的検定
def statistical_tests(df):
    """統計的検定の実行"""
    spread_columns = ['M1_M2_spread', 'M2_M3_spread', 'M3_M4_spread']
    spread_names = ['M1-M2', 'M2-M3', 'M3-M4']
    
    print("🔬 統計的検定結果:")
    print("=" * 60)
    
    test_results = {}
    
    for col, name in zip(spread_columns, spread_names):
        data = df[col].dropna()
        
        # Shapiro-Wilk正規性検定
        if len(data) <= 5000:  # サンプル数制限
            shapiro_stat, shapiro_p = stats.shapiro(data)
        else:
            shapiro_stat, shapiro_p = stats.shapiro(data.sample(5000))
        
        # Jarque-Bera正規性検定
        jb_stat, jb_p = stats.jarque_bera(data)
        
        # Kolmogorov-Smirnov正規性検定
        ks_stat, ks_p = stats.kstest(data, 'norm', args=(data.mean(), data.std()))
        
        # Augmented Dickey-Fuller単位根検定（定常性）
        from statsmodels.tsa.stattools import adfuller
        adf_result = adfuller(data)
        
        test_results[name] = {
            'shapiro': (shapiro_stat, shapiro_p),
            'jarque_bera': (jb_stat, jb_p),
            'ks_test': (ks_stat, ks_p),
            'adf': adf_result
        }
        
        print(f"\n{name}スプレッド:")
        print(f"  Shapiro-Wilk正規性検定: 統計量={shapiro_stat:.4f}, p値={shapiro_p:.4f}")
        print(f"  Jarque-Bera正規性検定: 統計量={jb_stat:.4f}, p値={jb_p:.4f}")
        print(f"  K-S正規性検定: 統計量={ks_stat:.4f}, p値={ks_p:.4f}")
        print(f"  ADF単位根検定: 統計量={adf_result[0]:.4f}, p値={adf_result[1]:.4f}")
        
        # 解釈
        normal_tests = [shapiro_p, jb_p, ks_p]
        if all(p < 0.05 for p in normal_tests):
            print(f"  → 正規分布ではない（全検定でp<0.05）")
        elif any(p >= 0.05 for p in normal_tests):
            print(f"  → 正規分布の可能性あり（一部検定でp≥0.05）")
        
        if adf_result[1] < 0.05:
            print(f"  → 定常系列（ADF検定p<0.05）")
        else:
            print(f"  → 非定常系列の可能性（ADF検定p≥0.05）")
    
    return test_results

# 統計的検定実行
test_results = statistical_tests(spreads_data)

## 5. 季節性とパターン分析

In [ ]:
# 季節性分析
def analyze_seasonality(df):
    """季節性とパターン分析"""
    seasonal_df = df.copy()
    
    # 時間的特徴量を追加
    seasonal_df['year'] = seasonal_df.index.year
    seasonal_df['month'] = seasonal_df.index.month
    seasonal_df['quarter'] = seasonal_df.index.quarter
    seasonal_df['day_of_year'] = seasonal_df.index.dayofyear
    seasonal_df['weekday'] = seasonal_df.index.weekday
    
    return seasonal_df

def plot_seasonality_analysis(df):
    """季節性分析チャート"""
    seasonal_data = analyze_seasonality(df)
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'M1-M2スプレッド 月別平均',
            'M1-M2スプレッド 四半期別分布',
            'M1-M2スプレッド 年別推移',
            '曜日別スプレッド特性'
        )
    )
    
    # 1. 月別平均
    monthly_avg = seasonal_data.groupby('month')['M1_M2_spread'].agg(['mean', 'std']).reset_index()
    
    fig.add_trace(
        go.Scatter(
            x=monthly_avg['month'],
            y=monthly_avg['mean'],
            error_y=dict(type='data', array=monthly_avg['std']),
            mode='lines+markers',
            name='月別平均',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    
    # 2. 四半期別ボックスプロット
    for quarter in [1, 2, 3, 4]:
        quarter_data = seasonal_data[seasonal_data['quarter'] == quarter]['M1_M2_spread']
        fig.add_trace(
            go.Box(
                y=quarter_data,
                name=f'Q{quarter}',
                boxpoints='outliers'
            ),
            row=1, col=2
        )
    
    # 3. 年別平均推移
    yearly_avg = seasonal_data.groupby('year')['M1_M2_spread'].agg(['mean', 'std']).reset_index()
    
    fig.add_trace(
        go.Scatter(
            x=yearly_avg['year'],
            y=yearly_avg['mean'],
            error_y=dict(type='data', array=yearly_avg['std']),
            mode='lines+markers',
            name='年別平均',
            line=dict(color='red')
        ),
        row=2, col=1
    )
    
    # 4. 曜日別分析
    weekday_avg = seasonal_data.groupby('weekday')['M1_M2_spread'].agg(['mean', 'std']).reset_index()
    weekday_names = ['月', '火', '水', '木', '金', '土', '日']
    weekday_avg['weekday_name'] = [weekday_names[i] for i in weekday_avg['weekday']]
    
    fig.add_trace(
        go.Bar(
            x=weekday_avg['weekday_name'],
            y=weekday_avg['mean'],
            error_y=dict(type='data', array=weekday_avg['std']),
            name='曜日別平均',
            marker_color='green'
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        title=dict(
            text="M1-M2スプレッド 季節性・パターン分析",
            x=0.5,
            font=dict(size=16)
        ),
        height=700,
        showlegend=False
    )
    
    # 軸ラベル更新
    fig.update_xaxes(title_text="月", row=1, col=1)
    fig.update_yaxes(title_text="平均スプレッド (USD/t)", row=1, col=1)
    fig.update_yaxes(title_text="スプレッド (USD/t)", row=1, col=2)
    fig.update_xaxes(title_text="年", row=2, col=1)
    fig.update_yaxes(title_text="平均スプレッド (USD/t)", row=2, col=1)
    fig.update_xaxes(title_text="曜日", row=2, col=2)
    fig.update_yaxes(title_text="平均スプレッド (USD/t)", row=2, col=2)
    
    return fig, seasonal_data

# 季節性分析実行
seasonality_chart, seasonal_data = plot_seasonality_analysis(spreads_data)
seasonality_chart.show()

# 画像保存
seasonality_chart.write_image('../generated_images/adjacent_spreads_seasonality_analysis.png', 
                             width=1200, height=700, scale=2)

In [ ]:
# 季節性の統計的有意性検定
def test_seasonality_significance(seasonal_data):
    """季節性の統計的有意性を検定"""
    print("🗓️ 季節性統計的検定:")
    print("=" * 50)
    
    # ANOVA検定（月別グループ間差異）
    monthly_groups = [seasonal_data[seasonal_data['month'] == i]['M1_M2_spread'].dropna() 
                     for i in range(1, 13)]
    
    f_stat, p_value = stats.f_oneway(*monthly_groups)
    print(f"月別ANOVA検定:")
    print(f"  F統計量: {f_stat:.4f}")
    print(f"  p値: {p_value:.4f}")
    print(f"  結果: {'月別に有意差あり' if p_value < 0.05 else '月別に有意差なし'}")
    
    # 四半期別ANOVA検定
    quarterly_groups = [seasonal_data[seasonal_data['quarter'] == i]['M1_M2_spread'].dropna() 
                       for i in range(1, 5)]
    
    f_stat_q, p_value_q = stats.f_oneway(*quarterly_groups)
    print(f"\n四半期別ANOVA検定:")
    print(f"  F統計量: {f_stat_q:.4f}")
    print(f"  p値: {p_value_q:.4f}")
    print(f"  結果: {'四半期別に有意差あり' if p_value_q < 0.05 else '四半期別に有意差なし'}")
    
    # 曜日別ANOVA検定
    weekday_groups = [seasonal_data[seasonal_data['weekday'] == i]['M1_M2_spread'].dropna() 
                     for i in range(7)]
    
    f_stat_w, p_value_w = stats.f_oneway(*weekday_groups)
    print(f"\n曜日別ANOVA検定:")
    print(f"  F統計量: {f_stat_w:.4f}")
    print(f"  p値: {p_value_w:.4f}")
    print(f"  結果: {'曜日別に有意差あり' if p_value_w < 0.05 else '曜日別に有意差なし'}")
    
    # 月別統計サマリー
    print(f"\n📈 月別統計サマリー:")
    monthly_stats = seasonal_data.groupby('month')['M1_M2_spread'].agg([
        'count', 'mean', 'std', 'min', 'max'
    ]).round(4)
    
    monthly_stats.index = [f'{i}月' for i in monthly_stats.index]
    print(monthly_stats)
    
    return {
        'monthly_anova': (f_stat, p_value),
        'quarterly_anova': (f_stat_q, p_value_q),
        'weekday_anova': (f_stat_w, p_value_w),
        'monthly_stats': monthly_stats
    }

# 季節性有意性検定実行
seasonality_tests = test_seasonality_significance(seasonal_data)

## 6. 分析結果サマリーと洞察

In [ ]:
# 包括的分析サマリー
def generate_analysis_summary(spreads_data, basic_stats, seasonality_tests):
    """包括的分析サマリーの生成"""
    
    print("📋 LME銅先物 隣月間スプレッド分析サマリー")
    print("=" * 70)
    
    print(f"\n🔢 データ概要:")
    print(f"  分析期間: {spreads_data.index.min().strftime('%Y-%m-%d')} ～ {spreads_data.index.max().strftime('%Y-%m-%d')}")
    print(f"  データポイント数: {len(spreads_data):,}")
    print(f"  分析対象: M1-M2, M2-M3, M3-M4 隣月間スプレッド")
    
    print(f"\n📊 基本統計特性:")
    for col in ['M1-M2', 'M2-M3', 'M3-M4']:
        mean_val = basic_stats.loc['平均', col]
        std_val = basic_stats.loc['標準偏差', col]
        skew_val = basic_stats.loc['歪度', col]
        
        print(f"  {col}スプレッド:")
        print(f"    平均: {mean_val:.2f} USD/t")
        print(f"    標準偏差: {std_val:.2f} USD/t")
        print(f"    歪度: {skew_val:.3f} ({'右歪み' if skew_val > 0 else '左歪み' if skew_val < 0 else '対称'})") 
    
    print(f"\n🎯 主要発見事項:")
    
    # スプレッド水準の特徴
    m1_m2_positive = (spreads_data['M1_M2_spread'] > 0).mean() * 100
    m2_m3_positive = (spreads_data['M2_M3_spread'] > 0).mean() * 100
    m3_m4_positive = (spreads_data['M3_M4_spread'] > 0).mean() * 100
    
    print(f"  1. スプレッド方向性:")
    print(f"     M1-M2: {m1_m2_positive:.1f}%が正値（コンタンゴ傾向）")
    print(f"     M2-M3: {m2_m3_positive:.1f}%が正値（コンタンゴ傾向）")
    print(f"     M3-M4: {m3_m4_positive:.1f}%が正値（コンタンゴ傾向）")
    
    # 相関関係
    corr_m1m2_m2m3 = spreads_data['M1_M2_spread'].corr(spreads_data['M2_M3_spread'])
    corr_m2m3_m3m4 = spreads_data['M2_M3_spread'].corr(spreads_data['M3_M4_spread'])
    
    print(f"\n  2. スプレッド間相関:")
    print(f"     M1-M2 vs M2-M3: {corr_m1m2_m2m3:.3f}")
    print(f"     M2-M3 vs M3-M4: {corr_m2m3_m3m4:.3f}")
    
    # 季節性
    monthly_anova_p = seasonality_tests['monthly_anova'][1]
    print(f"\n  3. 季節性:")
    print(f"     月別有意差: {'あり' if monthly_anova_p < 0.05 else 'なし'} (p={monthly_anova_p:.4f})")
    
    # 最も変動が大きい期間
    spreads_data_with_vol = spreads_data.copy()
    spreads_data_with_vol['year'] = spreads_data_with_vol.index.year
    yearly_vol = spreads_data_with_vol.groupby('year')['M1_M2_spread'].std()
    high_vol_year = yearly_vol.idxmax()
    low_vol_year = yearly_vol.idxmin()
    
    print(f"\n  4. ボラティリティ特性:")
    print(f"     最高ボラティリティ年: {high_vol_year} ({yearly_vol[high_vol_year]:.2f})")
    print(f"     最低ボラティリティ年: {low_vol_year} ({yearly_vol[low_vol_year]:.2f})")
    
    print(f"\n💡 投資・取引への示唆:")
    print(f"  • 隣月間スプレッドは主にコンタンゴ状態で推移")
    print(f"  • M1-M2スプレッドが最も変動が大きく、流動性も高い")
    print(f"  • スプレッド間に一定の相関があり、ポートフォリオ効果の検討が必要")
    
    if monthly_anova_p < 0.05:
        best_month = seasonality_tests['monthly_stats']['mean'].idxmax()
        worst_month = seasonality_tests['monthly_stats']['mean'].idxmin()
        print(f"  • 季節性あり: {best_month}が最高、{worst_month}が最低")
    
    print(f"  • リスク管理には標準偏差の2-3倍をストップロス水準として設定推奨")
    
    return {
        'data_period': (spreads_data.index.min(), spreads_data.index.max()),
        'sample_size': len(spreads_data),
        'spread_characteristics': {
            'M1_M2_positive_ratio': m1_m2_positive,
            'M2_M3_positive_ratio': m2_m3_positive,
            'M3_M4_positive_ratio': m3_m4_positive
        },
        'correlations': {
            'M1M2_vs_M2M3': corr_m1m2_m2m3,
            'M2M3_vs_M3M4': corr_m2m3_m3m4
        },
        'seasonality_significant': monthly_anova_p < 0.05,
        'volatility_years': {
            'highest': (high_vol_year, yearly_vol[high_vol_year]),
            'lowest': (low_vol_year, yearly_vol[low_vol_year])
        }
    }

# サマリー生成
analysis_summary = generate_analysis_summary(spreads_data, basic_stats, seasonality_tests)

In [ ]:
# 分析結果をCSVで保存
def save_analysis_results(spreads_data, basic_stats, analysis_summary):
    """分析結果をファイルに保存"""
    
    # 出力ディレクトリ作成
    os.makedirs('../analysis_results/adjacent_spreads', exist_ok=True)
    
    # 1. 基本統計量
    basic_stats.to_csv('../analysis_results/adjacent_spreads/basic_statistics.csv', encoding='utf-8-sig')
    
    # 2. 時系列データ
    spreads_data.to_csv('../analysis_results/adjacent_spreads/spread_timeseries.csv', encoding='utf-8-sig')
    
    # 3. 日次リターン
    returns_data = spreads_data[['M1_M2_spread_change', 'M2_M3_spread_change', 'M3_M4_spread_change']].copy()
    returns_data.to_csv('../analysis_results/adjacent_spreads/spread_returns.csv', encoding='utf-8-sig')
    
    # 4. 相関行列
    correlation_matrix = spreads_data[['M1_M2_spread', 'M2_M3_spread', 'M3_M4_spread']].corr()
    correlation_matrix.to_csv('../analysis_results/adjacent_spreads/correlation_matrix.csv', encoding='utf-8-sig')
    
    # 5. 分析サマリー（JSON）
    import json
    
    # 日付をJSON serializable形式に変換
    summary_for_json = analysis_summary.copy()
    summary_for_json['data_period'] = [
        analysis_summary['data_period'][0].strftime('%Y-%m-%d'),
        analysis_summary['data_period'][1].strftime('%Y-%m-%d')
    ]
    
    with open('../analysis_results/adjacent_spreads/analysis_summary.json', 'w', encoding='utf-8') as f:
        json.dump(summary_for_json, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 分析結果を保存しました:")
    print(f"  📈 基本統計量: ../analysis_results/adjacent_spreads/basic_statistics.csv")
    print(f"  📊 時系列データ: ../analysis_results/adjacent_spreads/spread_timeseries.csv")
    print(f"  📉 日次リターン: ../analysis_results/adjacent_spreads/spread_returns.csv")
    print(f"  🔗 相関行列: ../analysis_results/adjacent_spreads/correlation_matrix.csv")
    print(f"  📋 分析サマリー: ../analysis_results/adjacent_spreads/analysis_summary.json")

# 分析結果保存
save_analysis_results(spreads_data, basic_stats, analysis_summary)

## 次のステップ

この基本分析により、LME銅先物の隣月間スプレッドの特性を把握しました。

### 主要発見事項
1. **スプレッド方向性**: 主にコンタンゴ状態で推移
2. **相関構造**: 隣接スプレッド間に一定の相関関係
3. **季節性**: 統計的に有意な月別パターンの存在
4. **リスク特性**: 年によってボラティリティに大きな差

### 次の分析ステップ
1. **高度な時系列分析**: ARIMA/GARCHモデルによる予測
2. **機械学習モデル**: より複雑なパターン認識
3. **取引戦略開発**: 統計的裁定とペアトレード
4. **リスク管理**: VaRとストレステスト
5. **バックテスト**: 実際の取引戦略の検証

次のノートブック `2_adjacent_spreads_correlation_analysis.ipynb` で、より詳細な相関分析と共和分テストを実施します。